<div id="Fundamentals" style="color:#37475a; border-bottom: 7px solid orange; width: 100%; margin-bottom: 15px; padding-bottom: 2px">
  <h2><font size="5">PHASE 3: Prediction of New Samples (Clients)</font></h2>
</div>


In [1]:
from time import time
from datetime import datetime, timedelta
import pandas as pd  
import numpy as np  
from matplotlib import*
from matplotlib.cm import register_cmap
import matplotlib.pyplot as plt 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, MinMaxScaler
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import Pipeline
import pickle
import seaborn as sns
import copy

from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier #KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
#from keras.layers import Dropout

from sklearn import metrics 
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from scipy import stats
import seaborn
from sklearn import set_config                      
print('Módulos importados')

Módulos importados


In [10]:
#Se carga el pipeline de transformación
def cargarPipeline(nombreArchivo):
    with open(nombreArchivo+'.pickle', 'rb') as handle:
        pipeline = pickle.load(handle)
    return pipeline

#Se carga el modelo entrenado
def cargarModelo(nombreArchivo):
    with open(nombreArchivo+'.pickle', 'rb') as handle:
        pipeline = pickle.load(handle)
    return pipeline

#Esta es la función para calcular la certeza (confianza o probabilidad) asociada a la predicción de clase
def obtenerResultadosyCertezas(lista):
    predicciones=lista
    marcas=[]
    certezas=[]
    nuevomax=1
    nuevomin=0
    marca=-1
    certeza=-1
    for i in range(len(lista)):
        prediccion=lista[i]
        if (prediccion < 0.5):
            marca = 'Sin Enfermedad'
            maxa=0.5
            mina=0
            certeza=1-((prediccion-mina)/(maxa-mina)*(nuevomax-nuevomin)+nuevomin)
            certeza=str(int((certeza)*100))+'%'
        elif (prediccion >= 0.5):
            marca = 'Enfermo'
            maxa=1
            mina=0.5
            certeza=(prediccion-mina)/(maxa-mina)*(nuevomax-nuevomin)+nuevomin
            certeza=str(int((certeza)*100))+'%'
        marcas.append(marca)
        certezas.append(certeza)
    return prediccion, marcas, certezas

def predecirNuevoPaciente(age: 53.0, sex: 1.0, cp: 0.0, trestbps: 140.0, chol: 203.0, fbs: 1.0, 
                          restecg: 0.0, thalach: 155.0, exang: 1.0, oldpeak: 3.1, slope: 0.0, 
                          ca: 0.0, thal: 3.0,):    
    cnames=['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
            'exang', 'oldpeak', 'slope', 'ca', 'thal']
    
    Xnew=[age, sex, cp, trestbps, chol, fbs, restecg, thalach,
            exang, oldpeak, slope, ca, thal]
    Xnew_Dataframe = pd.DataFrame(data=[Xnew],columns=cnames)
    pipe=cargarPipeline("pipePreprocesadores")
    Xnew_Transformado=pipe.transform(Xnew_Dataframe)
    modelo=cargarModelo("modeloKNN")
    y_pred=modelo.predict(Xnew_Transformado)
    predicciones, marcas, certezas= obtenerResultadosyCertezas(y_pred)
    dataframeFinal=pd.DataFrame({'Predicción':predicciones, 'Resultado':marcas, 'Certeza': certezas})
    np.set_printoptions(formatter={'float': lambda X: "{0:0.0f}".format(X)})
    return dataframeFinal

In [16]:
dataframeFinal = predecirNuevoPaciente(age=52.0, sex=1.0, cp=0.0, trestbps=125.0, chol=212.0, fbs=0.0, restecg=1.0, thalach=168.0, exang=0.0, oldpeak=1.0, slope=2.0, ca=2.0, thal=3.0)
dataframeFinal.head()

,Predicción,Resultado,Certeza
0,0.28,Sin Enfermedad,43%


In [17]:
dataframeFinal = predecirNuevoPaciente(age=71.0, sex=0.0, cp=0.0, trestbps=112.0, chol=149.0, fbs=0.0, restecg=1.0, thalach=125.0, exang=0.0, oldpeak=1.6, slope=1.0, ca=0.0, thal=2.0)
dataframeFinal.head()

,Predicción,Resultado,Certeza
0,1.0,Enfermo,100%


In [18]:
dataframeFinal = predecirNuevoPaciente(age=34.0, sex=0.0, cp=1.0, trestbps=118.0, chol=210.0, fbs=0.0, restecg=1.0, thalach=192.0, exang=0.0, oldpeak=0.7, slope=2.0, ca=0.0, thal=2.0)
dataframeFinal.head()

,Predicción,Resultado,Certeza
0,1.0,Enfermo,100%


In [19]:
dataframeFinal = predecirNuevoPaciente(age=51.0, sex=1.0, cp=0.0, trestbps=140.0, chol=298.0, fbs=0.0, restecg=1.0, thalach=122.0, exang=1.0, oldpeak=4.2, slope=1.0, ca=3.0, thal=3.0)
dataframeFinal.head()

,Predicción,Resultado,Certeza
0,0.0,Sin Enfermedad,100%


In [20]:
dataframeFinal = predecirNuevoPaciente(age=58.0, sex=1.0, cp=2.0, trestbps=140.0, chol=211.0, fbs=1.0, restecg=0.0, thalach=165.0, exang=0.0, oldpeak=0.0, slope=2.0, ca=0.0, thal=2.0)

dataframeFinal.head()

,Predicción,Resultado,Certeza
0,0.88,Enfermo,76%
